Starting with guidance from Dan's notebook (http://dan.iel.fm/posts/exopop/), let's try to expand the occurrence model to a third dimension (dilution).

In [1]:
from __future__ import division, print_function
import os
import requests
import pandas as pd
from cStringIO import StringIO

def get_catalog(name, basepath="data"):
    fn = os.path.join(basepath, "{0}.h5".format(name))
    if os.path.exists(fn):
        return pd.read_hdf(fn, name)
    if not os.path.exists(basepath):
        os.makedirs(basepath)
    print("Downloading {0}...".format(name))
    url = ("http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/"
           "nph-nstedAPI?table={0}&select=*").format(name)
    r = requests.get(url)
    if r.status_code != requests.codes.ok:
        r.raise_for_status()
    fh = StringIO(r.content)
    df = pd.read_csv(fh)
    df.to_hdf(fn, name, format="t")
    return df

In [6]:
stlr = get_catalog('q1_q17_dr24_stellar')

In [7]:
kois = get_catalog('q1_q17_dr24_koi')

OK, for any given star, we want to calculate the detection efficiency as a function of $P$, $R_p$, **and** a dilution factor $f_d$.  Let's define $f_d$ as the fraction of light in the aperture that does *not* belong to the target star.  This, the true radius of a planet $R_p$ is 

$$\frac{R_{p,0}}{\sqrt{1 - f_d}}$$

where $R_{p,0}$ is the planet "radius" assuming no dilution.  Thus, if a 